# StoNED with Z variables

A firm’s ability to operate efficiently often depends on operational conditions 
and practices, such as the production environment and the firm specific 
characteristics for  example  technology  selection  or  managerial  
practices.  Banker  and  Natarajan (2008) refer to both variables that 
characterize operational conditions and practices as `contextual variables`.

* Contextual variables are often (but not always) **external factors** that are beyond the control of firms

    - Examples: competition, regulation, weather, location

    - Need to adjust efficiency estimates for operating environment

    - Policy makers may influence the operating environment

* Contextual variables can also be **internal factors**

    - Examples: management practices, ownership
    
    - Better understanding of the impacts of internal factors can help the firm to improve performance


Taking the multiplicative model as our starting point, 
we introduce the contextual variables, represented by `r`-dimensional vectorszi that 
represent the measured values of operational conditions and practices, to obtain 
the following semi-nonparametric, partial log-linear equation


   \begin{align*}
        \text{ln} y_i = \text{ln} f(\bf x_i) + \delta^{'}Z_i + v_i - u_i.
   \end{align*}

In this equation, parameter vector $\delta=(\delta_1...\delta_r)$ represents the 
marginal effects ofcontextual variables on output. All other variables maintain their 
previous definitions.

Following Johnson and Kuosmanen (2011), we incorporate the contextual variables in step 1 of 
the StoNED estimation routine and refine the multiplicative CNLS problem as follows:

   \begin{align*}
        & \underset{\alpha, \beta, \varepsilon} {min} \sum_{i=1}^n\varepsilon_i^2 \\
        & \text{s.t.} \\
        &  \text{ln} y_i = \text{ln}(\phi_i+1) + \delta^{'}z_i + \varepsilon_i  \quad \forall i\\
        &   \phi_i  = \alpha_i + \beta_i^{'}X_i -1 \quad \forall i \\
        &  \alpha_i + \beta_i^{'}X_i \le \alpha_j + \beta_j^{'}X_i  \quad  \forall i, j\\
        &  \beta_i \ge 0 \quad  \forall i \\
   \end{align*}

Denote by $\delta^{StoNEZD}$ the coefficients  of  the contextual variables 
obtained as theoptimal solution to above nonlinear problem. Johnson and Kuosmanen (2011) examine the statisticalproperties of this estimator in 
detail, showing its unbiasedness, consistency, and asymptotic efficiency.

In [1]:
# import packages
from pystoned import StoNED
import pandas as pd
import numpy as np

In [3]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED/master/sources/data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head(5)

,OPEX,CAPEX,TOTEX,Energy,Length,Customers,PerUndGr
0,681,729,1612,75,878,4933,0.11
1,559,673,1659,62,964,6149,0.21
2,836,851,1708,78,676,6098,0.75
3,7559,8384,18918,683,12522,55226,0.13
4,424,562,1167,27,697,1670,0.03


In [4]:
# output (total cost)
y  = df['TOTEX']

# inputs 
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)

In [5]:
# Z variables
z = df['PerUndGr']

In [6]:
# define and solve the StoNED model using MoM approach
model = StoNED.StoNED(y, x, z, cet = "mult", fun = "cost", rts = "crs")
model.optimize(remote=True)

Estimating the multiplicative model remotely with knitro solver


In [7]:
# retrive the technical inefficiency
print(model.get_technical_inefficiency(method='MOM'))

[1.14019318 1.12019573 1.12853371 1.10526798 1.21740693 1.07147982
 1.10248408 1.19500296 1.24581116 1.078823   1.12411435 1.08170165
 1.23855484 1.24384493 1.08746003 1.07471726 1.19236034 1.26689226
 1.11583279 1.07649198 1.08401315 1.04319945 1.27664629 1.0732699
 1.19371439 1.11734768 1.24774948 1.054387   1.21435035 1.09990317
 1.09002821 1.04117802 1.09597759 1.24697686 1.0998877  1.19216685
 1.05325707 1.05718105 1.06576915 1.10371384 1.10870425 1.11148136
 1.16710793 1.07436745 1.07087126 1.06626849 1.12663516 1.1606251
 1.06619533 1.06140358 1.25659629 1.12366326 1.18023456 1.22919845
 1.1418339  1.05073495 1.06045053 1.09172299 1.05397693 1.0961019
 1.06366561 1.08680063 1.09093216 1.12997678 1.29097093 1.08062683
 1.17623298 1.08235818 1.12862379 1.04294507 1.09829237 1.12144504
 1.06496089 1.07869276 1.06874922 1.09741481 1.10745816 1.18109028
 1.09573569 1.08563667 1.09999072 1.07166877 1.06815367 1.12502119
 1.11625943 1.11815312 1.13553093 1.30889362 1.13641974]
